In [112]:
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline, AutoModel

from sklearn.metrics.pairwise import cosine_similarity

from datasets import load_dataset
import math
import json

import numpy as np


In [134]:
text1 = "Válka na Ukrajině je konflikt, který začal 24. února 2022 rozsáhlou invazí Ruských vojsk na Ukrajinu ve snaze ovládnout zemi během několika dní. Ukrajina se zvládla úspěšně bránit a částečně zatlačit ruskou armádu zpět. Mnoho evropských zemí a USA ruský útok odsoudili a pomáhají Ukrajině s dovozem zbraní, humanitární pomocí a také s přijímáním mnoha uprchlíků z Ukrajiny. Na Rusko bylo uvaleno mnoho ekonomických sankcí. Rusko rovněž vede hybridní válku šířením lží po internetu prostřednictvím sociálních sítí, zejména v regionu střední Evropy. Ruská agrese přesvědčila neutrální země jako Finsko a Švédsko, aby vstoupily do NATO. Ukrajinu ve válce od začátku vede prezident Zelenenský. Na druhé straně byl ruský prezident Vladimír Putin. Ruské i ukrajinské síly vytvořily řadu zákopových linií na obranu zabraných území, což znamenalo k zpomalení ofenzívy. I když prezident Zelenskyj projevuje svou vděčnost západním zemím, stále říká, že Ukrajina dostává jen tolik zbraní, aby se bránila, ale ne tolik, aby porazila Rusko. Západní svět se bojí jaderných zbraní, kterými Rusko disponuje. Premiér Petr Fiala schvaluje pomoc, která je Ukrajině posílána. Byl jedním z prvních evropských politiků, kteří navštívili prezidenta Zelenského v Kyjevě na Ukrajině. Současná česká vláda je stále jednou z nejvíce proukrajinských a protiruských vlád na světě, ačkoliv popularita podpory ukrajiny ve společnosti klesá. Petr Fiala je kritizován za podporu ukrajinců a ukrajinských uprchlíků na úkor českých občanů. Ten to však odmítá. Je zastánce zvyšování rozpočtu armády a ministerstva obrany. Za jeho vlády se nakoupili nové stíhačky F-35. Fialovi odpůrci nákup kritizují jako rozpočtově nezodpovědný." 

text2 = "Konflikt mezi Izraelem a Hamásem začal 7. října 2023, kdy byla z pásma Gazy vypálena palba tisíců raket namířených na Izrael a asi 3000 ozbrojených teroristů prolomilo zeď oddělující Izrael a Gazu a zaútočilo na nedaleké vesnice, rabovali, znásilňovali a zabíjeli. Zpátky s sebou vzali stovky rukojmí zpět do Gazy. Izrael odpověděl odříznutím pásma Gazy od vody a zásob a začal na Gazu odpalovat rakety. O několik týdnů později Izrael zahájil invazi do pásma Gazy. Konflikt narušuje regionální stabilitu. Hamás se snaží vyměnit rukojmí za Palestince držené v izraelských věznicích. Koncem listopadu se na několik dní konalo příměří. V Evropě a Asii se konalo mnoho protestů za palestinský lid, aby Izrael zastavil válku a pustil do Gazy humanitární pomoc. Dosud Izrael vpustil určitou pomoc od mnoha mezinárodních humanitárních organizací. Do dnes zemřelo v palestině přes deset tisíc lidí, převážně dětí. Společenská podpora izraele v České republice je velmi velká. Petr Fiala vyjádřil podporu izraelské vládě a zdůraznil právo izraelského lidu na sebeobranu. Česká republika je dlouhodobý partner Israele a Petr Fiala je odhodlán v této tradici dále pokračovat. Hamás označuje za teroristickou organizaci. Levicoví odpůrci Petra Fialy ho kritizují za ignorování civilních obětí způsobených Izraelským bombardováním pásma Gazy." 

text3 = "Prezidentské volby se v České republice konaly v lednu 2023. První kolo vyhrál Petr Pavel, bývalý předseda Vojenského výboru NATO, Petr Pavel byl proNATO, proevropský a prozápadní kandidát. Premiér Petr Fiala projevil podporu Petru Pavlovi. Těsně na druhém místě skončil populistický kandidát, bývalý premiér Andrej Babiš, který vyjádřil nesouhlas s českou účastí ve válce na Ukrajině a s neschopností vlády starat se o lidi. Druhé kolo se konalo 27. a 28. ledna. Andrej Babiš byl odpůrce Petra Fialy. Petr Pavel vyhrál druhé kolo proti Babišovi se značným náskokem hlasů a stal se nově zvoleným prezidentem České republiky. Do funkce se ujal na jaře 2023 a nahradil ve funkci Miloše Zemana. Ve volbách byla nejvyšší volební účast od roku 1998. Petr Fiala má s prezidentem Petrem Pavlem sdílí prozápadní hodnoty, názory na Ukrajinu, která byla napadena Ruskem. Koalici SPOLU vedená Petrem Fialou podpořila Pavlovu kandidaturu." 

text4 = "Slovenské parlamentní volby se konaly v září 2023 poté, co předchozí vláda Igora Matoviče prohrála hlasování o nedůvěře. Vítěz voleb, levicový populista a proruský politik Robert Fico dokázal sestavit vládu se sociálně demokratickou stranou HLAS a Slovenskou národní stranou. Robert Fico vedl kampaň šířením dezinformací, lží a kritikou evropské vlády, Ukrajiny a myšlenek liberální demokracie. Během posledních pár let měli vyšetřovatelé plné ruce práce s odhalováním mnoha korupčních obchodů, které byly uskutečněny za Ficovy předchozí vlády. Petr Fiala s Ficem nesdílí názory na proruskou orientaci země. Po volební noci ale Ficovi pogratuloval a popřál Slovensku 'dobrou vládu' v naději, že Česká a Slovenská republika udrží své skvělé vztahy, což není zcela jisté. Hrozí obrat Slovenska na východ a ještě větší příklon k Rusku, které Petr Fiala kritizuje za vedení války na Ukrajině." 

text5 = "Petr Fiala, narozený 1. září 1964 v Brně je český politik, politolog a vysokoškolský učitel, od listopadu 2021 předseda vlády České republiky v čele většinového kabinetu tvořeného koaličními subjekty SPOLU a Piráti a Starostové. V poslanecké sněmovně zastupuje stranu ODS od roku 2013. Aktuálně je předsedou strany ODS a vede koalici SPOLU, složenou z pravicové ODS, křesťanskými Lidovci a stranou TOP 09. Působil jako ministr školství v Nečasově kabinetu v letech 2012 a 2013. Do roku 2011 byl rektorem a následně prorektorem Masarykovy univerzity v Brně.  Pochází z moravské měšťanské rodiny. Dědeček František Fiala byl právník, který za první republiky pracoval jako vrchní rada politické správy na okresních hejtmanstvích v Ostravě, Hodoníně a Olomouci, později pak jako státní rada v Brně. Babička Františka Fialová z otcovy strany byla židovka a část příbuzných byla za války deportována do německých koncentračních táborů. V roce 1992 se Petr Fiala oženil s Janou Fialovou s níž se seznámil během sametové revoluce. Manželka je bioložka, působící na Ústavu ochrany a podpory zdraví Lékařské fakulty Masarykovy univerzity. V květnu 2023 byla jmenována prorektorkou Masarykovy univerzity. Do manželství se narodily tři děti. Nejstarší Martin Fiala vystudoval dějiny umění na FF MU, Klára Fialová se stala posluchačkou lékařství na LF MU a nejmladší Jiří Fiala nastoupil ke studiu historie a ekonomie."  

text6 = "Mezi hlavní domácí problémy České republiky, s kterými se vládní koalice složená z koalic SPOLU a Pirátů a Starostů vedená premiérem Petrem Fialou, musí vypořádat patří rostoucí statní deficit. Státní dluh vzrostl na 3.2 bilionu korun. Petr Fiala se snaží snižovat výdaje státního rozpočtu a zároveň nezvyšovat daně, jak sliboval ve svém programu. Státní dluh je tvořen dluhy vlády a vzniká především hromaděním schodků státního rozpočtu. Financován je pokladničními poukázkami, státními dluhopisy, přímými půjčkami nebo půjčkami od Evropské investiční banky. Petr Fiala říká, že vláda vysoké zadlužení zdědila po vládě Andreje Babiše a jeho nezodpovědné fiskální politice. Fiala tvrdí, že Babiš zbytečně rozhazoval a během ekonomického růstu málo investoval. Další aktuální ekonomický a společenský problém je rostoucí chudoba a bída. Přes milion lidí je v exekucích a reálná mzda klesá. Petr Fiala odmítá, že by někoho nechala padnout. Prostřednictvím programu Milostivé léto se řada lidí s dluhy a exekucemi mělo možnost těchto dluhů zbavit. Zároveň se vláda snaží poskytovat sociální pomoc těm, kteří jí skutečně potřebují. Rovněž se snaží pomáhat uprchlíkům z Ukrajiny, kteří utekli z Ukrajiny po Ruské invazi." 

 

In [133]:
text1 = "The war in Ukraine is a conflict that began on February 24, 2022 with a large-scale invasion of Ukraine by Russian troops in an attempt to take control of the country within a few days. Ukraine managed to successfully defend itself and partially push back the Russian army. Many European countries and the US condemned the Russian attack and they are helping Ukraine with arms importation, humanitarian aid and also accepting many refugees from Ukraine. Many economic sanctions have been imposed on Russia. Russia is also waging a hybrid war by spreading lies on the Internet through social networks, especially in the Central Europe region. Russian aggression has convinced neutral countries like Finland and Sweden to join NATO. Ukraine has been led in the war from the beginning by President Zelenskyi. On the other side was Russian President Vladimir Putin. Both Russian and Ukrainian forces have created a series of trench lines to defend the captured territories, which has meant slowing down the offensive. Although President Zelenskyi shows his gratitude to Western countries, he keeps saying that Ukraine is getting only enough weapons to defend itself, but not enough to defeat Russia. The Western world is afraid of the nuclear weapons that Russia possesses. Prime Minister Petr Fiala approves the aid that is being sent to Ukraine. He was one of the first European politicians to visit President Zelensky in Kyiv, Ukraine. The current Czech government is still one of the most pro-Ukrainian and anti-Russian governments in the world, although the popularity of supporting Ukraine in society is declining. Petr Fiala is criticized for supporting Ukrainians and Ukrainian refugees at the expense of Czech citizens. However, he refuses. He is a supporter of increasing the budget of the army and the Ministry of Defense. New F-35 fighter jets were purchased during his administration. Fiala's opponents criticize the purchase as fiscally irresponsible."

text2 = "The conflict between Israel and Hamas began on October 7, 2023, when thousands of rockets were fired from the Gaza Strip aimed at Israel, and about 3,000 armed terrorists breached the wall separating Israel and Gaza and attacked nearby villages, looting, raping and killing. Back they took hundreds of hostages back to Gaza with them. Israel responded by cutting off Gaza from water and supplies and began firing rockets into Gaza. Weeks later, Israel launched an invasion of the Gaza Strip. The conflict is disrupting regional stability. Hamas seeks to exchange hostages for Palestinians held in Israeli prisons. A truce was held for several days at the end of November. There have been many protests in Europe and Asia for the Palestinian people for Israel to stop the war and allow humanitarian aid into Gaza. So far Israel has allowed some aid from many international humanitarian organizations. Until today, the death of over ten thousand people, mostly children, to Palestine. Social support for Israel in the Czech Republic is very large. Petr Fiala expressed support for the Israeli government and emphasized the Israeli people's right to self-defense. The Czech Republic is a long-term partner of Israel and Petr Fiala is determined to continue this tradition. He calls Hamas a terrorist organization. Petr Fiala's left-wing opponents criticize him for ignoring the civilian casualties caused by Israel's bombing of the Gaza Strip."

text3 = "Presidential elections were held in the Czech Republic in January 2023. The first round was won by Petr Pavel, the former chairman of the NATO Military Committee, Petr Pavel was a pro-NATO, pro-European and pro-Western candidate. Prime Minister Petr Fiala showed his support for Petr Pavel. The populist candidate, former Prime Minister Andrej Babiš, who expressed disapproval of Czech participation in the war in Ukraine and the government's inability to care for the people. The second round was held on January 27 and 28. Andrej Babiš was Petr Fiala's opponent. Petr Pavel won the second round against Babiš with a significant margin of votes and became the newly elected president of the Czech Republic. He took office in the spring of 2023 and replaced Miloš Zeman. The election saw the highest voter turnout since 1998. Petr Fiala and President Petr Pavlo share pro-Western values, views on Ukraine, which was attacked by Russia. The coalition TOGETHER led by Petr Fiala supported Pavlo's candidacy."

text4 = "Slovak parliamentary elections were held in September 2023 after the previous government of Igor Matovič lost a vote of no confidence. The winner of the election, left-wing populist and pro-Russian politician Robert Fico managed to form a government with the social democratic party HLAS and the Slovak National Party. Robert Fico led the campaign by spreading disinformation, lies and criticizing the European government, Ukraine and the ideas of liberal democracy. During the last few years, investigators have been busy uncovering many corrupt deals that were carried out under Fico's previous administrations. Petr Fiala and Fico do not share views on the country's pro-Russian orientation. After but on election night he congratulated Fico and wished Slovakia a 'good government' in the hope that the Czech and Slovak Republics will maintain their excellent relations, which is not entirely certain. Ukraine."

text5 = "Petr Fiala, born on September 1, 1964 in Brno, is a Czech politician, political scientist and university teacher, since November 2021 the Prime Minister of the Czech Republic at the head of the majority cabinet formed by the coalition entities SPOLU and Pirates and Starostové. He has represented the ODS party in the Chamber of Deputies since 2013. He is currently the chairman of the ODS party and leads the SPOLU coalition, composed of the right-wing ODS, Christian Lidovci and the TOP 09 party. He served as Minister of Education in the Nečas cabinet in 2012 and 2013. Until 2011, he was rector and then vice-rector of Masaryk University in Brno. He comes from a Moravian bourgeois family. Grandfather František Fiala was a lawyer who, during the First Republic, worked as a high councilor of the political administration at the district hetmanships in Ostrava, Hodonín and Olomouc, and later as a state councilor in Brno. Paternal grandmother Františka Fialová was Jewish and part relatives were deported to German concentration camps during the war.In 1992, Petr Fiala married Jana Fialová, whom he met during the Velvet Revolution. The wife is a biologist working at the Institute of Health Protection and Promotion of the Faculty of Medicine of Masaryk University. In May 2023, she was appointed vice-rector of Masaryk University. Three children were born to the marriage. The eldest, Martin Fiala, studied art history at the FF MU, Klára Fialová became a student of medicine at the LF MU, and the youngest Jiří Fiala started studying history and economics."

text6 = "Among the main domestic problems of the Czech Republic, which the government coalition consisting of SPOLU and the Pirates and Starost coalitions led by Prime Minister Petr Fiala, must deal with is the growing state deficit. The state debt has risen to 3.2 trillion crowns. Petr Fiala is trying to reduce state spending budget and at the same time not to raise taxes, as he promised in his program. The national debt is made up of the government's debts and arises mainly from the accumulation of state budget deficits. It is financed by treasury bills, government bonds, direct loans or loans from the European Investment Bank. Petr Fiala says that the government's high indebtedness inherited from the government of Andrej Babiš and his irresponsible fiscal policy. Fiala claims that Babiš squandered unnecessarily and underinvested during economic growth. Another current economic and social problem is growing poverty and misery. Over a million people are in foreclosures and the real wage is falling. Petr Fiala refuses to let anyone down.Through the Summer of Grace program, many people with debt and foreclosures have had the opportunity to get out of debt. At the same time, the government tries to provide social assistance to those who really need it. They are also trying to help refugees from Ukraine who fled Ukraine after the Russian invasion."


In [76]:
text1EMB = "Válka na Ukrajině je konflikt, který začal 24. února 2022 rozsáhlou invazí Ruských vojsk na Ukrajinu ve snaze ovládnout zemi během několika dní. Ukrajina se zvládla úspěšně bránit a částečně zatlačit ruskou armádu zpět. Mnoho evropských zemí a USA ruský vojenský útok odsoudili a pomáhají Ukrajině s dovozem zbraní, humanitární pomocí a také s přijímáním mnoha uprchlíků z Ukrajiny. Na Rusko bylo uvaleno mnoho ekonomických sankcí. Rusko rovněž vede hybridní válku šířením lží po internetu prostřednictvím sociálních sítí, zejména v regionu střední Evropy. Ruská agrese přesvědčila neutrální země jako Finsko a Švédsko, aby vstoupily do NATO. Ukrajinu ve válce od začátku vede prezident Zelenenský. Na druhé straně byl ruský prezident Vladimír Putin. Ruské i ukrajinské síly vytvořily řadu zákopových linií na obranu zabraných území, což znamenalo k zpomalení ofenzívy. I když prezident Zelenskyj projevuje svou vděčnost západním zemím, stále říká, že Ukrajina dostává jen tolik zbraní, aby se bránila, ale ne tolik, aby porazila Rusko. Západní svět se bojí jaderných zbraní, kterými Rusko disponuje. Premiér Petr Fiala schvaluje pomoc, která je Ukrajině posílána. Byl jedním z prvních evropských politiků, kteří navštívili prezidenta Zelenského v Kyjevě na Ukrajině. Současná česká vláda je stále jednou z nejvíce proukrajinských a protiruských vlád na světě, ačkoliv popularita podpory ukrajiny ve společnosti klesá. Petr Fiala je kritizován za podporu ukrajinců a ukrajinských uprchlíků na úkor českých občanů. Ten to však odmítá. Je zastánce zvyšování rozpočtu armády a ministerstva obrany. Za jeho vlády se nakoupili nové stíhačky F-35. Fialovi odpůrci nákup kritizují jako rozpočtově nezodpovědný. Jaký byl původní impulz k vyvolání konfliktu mezi Ruskem a Ukrajinou v únoru 2022? Jak se Ukrajině podařilo úspěšně bránit proti původní invazi ruských sil? Popište mezinárodní reakce na akce Ruska vůči Ukrajině. Jak podporovaly evropské země a Spojené státy Ukrajinu v reakci na ruskou invazi? Jaké ekonomické sankce byly uvaleny na Rusko v souvislosti s konfliktem? Prozkoumejte roli sociálních médií v hybridní válce, kterou vedlo Rusko ve střední Evropě. Diskutujte o vojenských strategiích, které byly použity jak ruskými, tak ukrajinskými silami, včetně vytváření zákopových linií pro obranu. Jaký byl vliv těchto zákopových linií na rychlost postupu ofenzívy? Jaký je postoj prezidenta Zelenského k západní pomoci a co říká o množství poskytovaných zbraní? Jaký je postoj ruského prezidenta Vladimíra Putina k průběhu války a mezinárodní reakci?  Jakým způsobem premiér Petr Fiala komentuje poskytovanou pomoc Ukrajině a jaký je jeho postoj k nákupu nových stíhaček F-35?  Jaký je současný postoj české veřejnosti k podpoře Ukrajiny, a jaký je trend popularity této podpory? Jaký je přístup premiéra Petra Fialy k kritice, že přiděluje podporu Ukrajině na úkor českých občanů?  Jak se staví kritika k nákupu nových stíhaček F-35 a co tvrdí její odpůrci ohledně rozpočtové zodpovědnosti? Ukrajina. Ukrajině. Ukrajiny. Rusko. Válka. Ruský. Rusáci. Rusákům. Válka. Operace. Speciální operace. Ukrajinci."  

  

text2EMB = "Konflikt mezi Izraelem a Hamásem začal 7. října 2023, kdy byla z pásma Gazy vypálena palba tisíců raket namířených na Izrael a asi 3000 ozbrojených teroristů prolomilo zeď oddělující Izrael a Gazu a zaútočilo na nedaleké vesnice, rabovali, znásilňovali a zabíjeli. Zpátky s sebou vzali stovky rukojmí zpět do Gazy. Izrael odpověděl odříznutím pásma Gazy od vody a zásob a začal na Gazu odpalovat rakety. O několik týdnů později Izrael zahájil invazi do pásma Gazy. Hamás se snaží vyměnit rukojmí za Palestince držené v izraelských věznicích. Koncem listopadu se na několik dní konalo příměří. V Evropě a Asii se konalo mnoho protestů za palestinský lid, aby Izrael zastavil válku a pustil do Gazy humanitární pomoc. Dosud Izrael vpustil určitou pomoc od mnoha mezinárodních humanitárních organizací. Do dnes zemřelo v palestině přes deset tisíc lidí, převážně dětí. Společenská podpora izraele v České republice je velmi velká. Petr Fiala vyjádřil podporu izraelské vládě a zdůraznil právo izraelského lidu na sebeobranu. Česká republika je dlouhodobý partner Israele a Petr Fiala je odhodlán v této tradici dále pokračovat. Hamás označuje za teroristickou organizaci. Levicoví odpůrci Petra Fialy ho kritizují za ignorování civilních obětí způsobených Izraelským bombardováním pásma Gazy. Řada zemí posílá Izraeli vojenskou pomoc. Kdy a jak začal konflikt mezi Izraelem a Hamásem v roce 2023? Jaká byla povaha prvních útoků ze strany Hamásu a jak Izrael na ně reagoval? Jaké byly důsledky izraelské invaze do pásma Gazy? Proč Hamás unesl stovky rukojmích při útoku na izraelské vesnice a jaká byla jejich osud?Jak Izrael odpověděl na palbu raket a jak se vyvíjel konflikt v následujících týdnech? Jaký byl cíl Izraele při odříznutí pásma Gazy od vody a zásob? Proč Hamás usiluje o výměnu rukojmích za Palestince ve věznicích v Izraeli? Jaký byl průběh příměří koncem listopadu a jak ovlivnil konflikt? Jaká je mezinárodní reakce na konflikt na blízkém východě, zejména v Evropě a Asii? Jaká je společenská podpora Izraele v České republice a jaký je postoj premiéra Petra Fialy k tomuto konfliktu? Proč Hamás označuje za teroristickou organizaci a jakou kritiku čelí od levicových odpůrců Petra Fialy? Jaký je dlouhodobý vztah České republiky k Izraeli a jak chce Petr Fiala pokračovat v této tradici? Jaká pomoc byla poskytnuta palestinskému lidu a jak je Izrael vnímán mezinárodními humanitárními organizacemi? Jaký je stav obětí konfliktu na palestinské straně a jak se vyvíjí situace v pásma Gazy? Palestina. Palestině. Izrael. Izraeli. Válka. Blízký východ. Irán. Arabové. Muslimové. Muslim. Židé. Židi. "  

  

text3EMB = "Prezidentské volby se v České republice konaly v lednu 2023. První kolo vyhrál Petr Pavel, bývalý předseda vojenského výboruNATO, Petr Pavel byl proNATO, proevropský a prozápadní kandidát. Premiér Petr Fiala projevil podporu Petru Pavlovi. Těsně na druhém místě skončil populistický kandidát, bývalý premiér Andrej Babiš, který vyjádřil nesouhlas s českou účastí ve válce na Ukrajině a s neschopností vlády starat se o lidi. Druhé kolo se konalo 27. a 28. ledna. Andrej Babiš byl odpůrce Petra Fialy. Petr Pavel vyhrál druhé kolo proti Babišovi se značným náskokem hlasů a stal se nově zvoleným prezidentem České republiky. Do funkce se ujal na jaře 2023 a nahradil ve funkci Miloše Zemana. Ve volbách byla nejvyšší volební účast od roku 1998. Petr Fiala má s prezidentem Petrem Pavlem sdílí prozápadní hodnoty, názory na Ukrajinu, která byla napadena Ruskem. Koalici SPOLU vedená Petrem Fialou podpořila Pavlovu kandidaturu. Jaké byly hlavní charakteristiky prezidentských voleb v České republice v lednu 2023? Kdo vyhrál první kolo voleb a jaké byly hlavní body jeho politického programu? Jaký postoj zaujal premiér Petr Fiala k vítězi prvního kola voleb? Kdo byl druhým nejúspěšnějším kandidátem v prvním kole a jaký byl jeho postoj k české účasti ve válce na Ukrajině? Jaké názory vyjádřil Andrej Babiš týkající se schopnosti vlády starat se o občany? Jak proběhlo druhé kolo voleb a jaký byl výsledek souboje mezi Petrem Pavlem a Andrejem Babišem? Jaké hodnoty a názory na Ukrajinu sdílí prezident Petr Pavel s premiérem Petrem Fialou? Jaká byla volební účast ve volbách a byly nějaké specifické aspekty tohoto volebního klání? Jaký byl postoj koalice SPOLU, vedené Petrem Fialou, k prezidentské kandidatuře Petra Pavla? Kdy Petr Pavel převzal prezidentský úřad a koho nahradil ve funkci prezidenta? Jaký je vztah mezi prezidentem Petrem Pavlem a předchozím prezidentem Milošem Zemanem? Jaké byly hlavní body politického programu prezidenta Petra Pavla a jak se liší od předchozího prezidenta? Jaká byla obecná reakce veřejnosti na výsledek voleb a změnu prezidenta? Jak se očekává, že se změna v prezidentském úřadu ovlivní českou politickou scénu a mezinárodní vztahy? Prezident. Prezidentský. Volby. Petr Pavel. Petr. Pavel."  

  

text4EMB = "Slovenské parlamentní volby se konaly v září 2023 poté, co předchozí vláda Igora Matoviče prohrála hlasování o nedůvěře. Vítěz voleb, levicový populista a proruský politik Robert Fico dokázal sestavit vládu se sociálně demokratickou stranou HLAS a Slovenskou národní stranou. Robert Fico vedl kampaň šířením dezinformací, lží a kritikou evropské vlády, Ukrajiny a myšlenek liberální demokracie. Během posledních pár let měli vyšetřovatelé plné ruce práce s odhalováním mnoha korupčních obchodů, které byly uskutečněny za Ficovy předchozí vlády. Petr Fiala s Ficem nesdílí názory na proruskou orientaci země. Po volební noci ale Ficovi pogratuloval a popřál Slovensku 'dobrou vládu' v naději, že Česká a Slovenská republika udrží své skvělé vztahy, což není zcela jisté. Hrozí obrat Slovenska na a ještě větší příklon k Rusku, které Petr Fiala kritizuje za vedení války na Ukrajině. Jaký byl důvod konání slovenských parlamentních voleb v září 2023 a jaké byly výsledky hlasování? Kdo byl vítězem voleb a jaká politická koalice se podařilo sestavit? Jakým způsobem Robert Fico vedl volební kampaň a s jakými tématy pracoval? Jaké byly hlavní prvky Ficovy kritiky evropské vlády, Ukrajiny a liberální demokracie během kampaně? Jak se Robert Fico vyrovnával s obviněními z korupce spojenými s jeho předchozí vládou? Jaký je postoj Petra Fialy k proruské orientaci Slovenska a jaké jsou obavy ohledně možného obratu k Rusku nebo Maďarsku? Jaký byl Petr Fialův postoj k vítězství Roberta Fica a co mu popřál po výsledcích voleb?Jaké jsou naděj  na udržení dobrých vztahů mezi Českou a Slovenskou republikou po změně ve slovenské vládě? Jaké jsou možné důsledky pro region, pokud Slovensko skutečně přeorientuje svou politiku k Rusku? Jak hodnotíte situaci, když levicový populistický politik s proruskými názory získává vedení ve vládě? Jaký může mít obrat Slovenska na východ dopad na stabilitu a bezpečnost v regionu? Jak se mohou tyto události projevit na postoji Slovenska k válce na Ukrajině a vztazích s Ruskem? Jaká je reakce veřejnosti na politiku a výsledky voleb na Slovensku? Jaký byl dopad této politické změny na mezinárodní scénu a vztahy mezi sousedními zeměmi ve střední Evropě? Fico. Matovič. Směr. Olano. Ficem."  

  

text5EMB = "Petr Fiala, narozený 1. září 1964 v Brně je český politik, politolog a vysokoškolský učitel, od listopadu 2021 předseda vlády České republiky v čele většinového kabinetu tvořeného koaličními subjekty SPOLU a Piráti a Starostové. V poslanecké sněmovně zastupuje stranu ODS od roku 2013. Aktuálně je předsedou strany ODS a vede koalici SPOLU, složenou z pravicové ODS, křesťanskými Lidovci a stranou TOP 09. Působil jako ministr školství v Nečasově kabinetu v letech 2012 a 2013. Do roku 2011 byl rektorem a následně prorektorem Masarykovy univerzity v Brně.  Pochází z moravské měšťanské rodiny. Dědeček František Fiala byl právník, který za první republiky pracoval jako vrchní rada politické správy na okresních hejtmanstvích v Ostravě, Hodoníně a Olomouci, později pak jako státní rada v Brně. Babička Františka Fialová z otcovy strany byla židovka a část příbuzných byla za války deportována do německých koncentračních táborů. V roce 1992 se Petr Fiala oženil s Janou Fialovou s níž se seznámil během sametové revoluce. Manželka je bioložka, působící na Ústavu ochrany a podpory zdraví Lékařské fakulty Masarykovy univerzity. V květnu 2023 byla jmenována prorektorkou Masarykovy univerzity. Do manželství se narodily tři děti. Nejstarší Martin Fiala vystudoval dějiny umění na FF MU, Klára Fialová se stala posluchačkou lékařství na LF MU a nejmladší Jiří Fiala nastoupil ke studiu historie a ekonomie. Jaké jsou klíčové životní údaje o Petrovi Fialovi, českém politikovi a předsedovi ODS? Kdy začal Petr Fiala svou politickou kariéru a jaké funkce zastával před předsednictvím vlády v roce 2021? Jaká politická spojenectví vedl Petr Fiala před svým předsednictvím vlády a jaká stranická koalice vytvořila jeho vláda v roce 2021? Jakou roli hrál Petr Fiala jako ministr školství v Nečasově kabinetu v letech 2012 a 2013? Jaký je jeho vztah k Masarykově univerzitě v Brně a jaké funkce zastával na této univerzitě? Jaký byl jeho rodinný původ a jaké jsou některé zajímavosti o jeho dědečkovi Františku Fialovi a babičce Františce Fialové? Jaká je historie manželství Petra Fialy s Janou Fialovou a jaké profese oba zastávají? Kdo jsou děti Petra Fialy a jaké obory studují? Jakým způsobem se rodina Petra Fialy angažuje ve společnosti a akademickém prostředí? Jaká byla reakce veřejnosti na jmenování Jany Fialové prorektorkou Masarykovy univerzity v květnu 2023? Jaký byl přínos Petra Fialy v oblasti politologie a jaké jsou jeho hlavní politické priority? Jaké jsou vyhlídky pro politickou kariéru Petra Fialy v budoucnosti? Jaký byl vývoj strany ODS pod vedením Petra Fialy a jak se promítá jeho politický styl do stranických aktivit? Jaké jsou hlavní výzvy, se kterými se Petr Fiala setkává ve své politické kariéře a v čele české vlády? Kdy ses narodil? Jak se jmenuje tvá žena? Jak se jmenují tvoje děti? Kam do školy chodí tvoje děti? Co jsi studoval?"   

  

text6EMB = "Mezi hlavní domácí problémy České republiky, s kterými se vládní koalice složená z koalic SPOLU a Pirátů a Starostů vedená premiérem Petrem Fialou, musí vypořádat patří rostoucí statní deficit. Státní dluh vzrostl na 3.2 bilionu korun. Petr Fiala se snaží snižovat výdaje státního rozpočtu a zároveň nezvyšovat daně, jak sliboval ve svém programu. Státní dluh je tvořen dluhy vlády a vzniká především hromaděním schodků státního rozpočtu. Financován je pokladničními poukázkami, státními dluhopisy, přímými půjčkami nebo půjčkami od Evropské investiční banky. Petr Fiala říká, že vláda vysoké zadlužení zdědila po vládě Andreje Babiše a jeho nezodpovědné fiskální politice. Fiala tvrdí, že Babiš zbytečně rozhazoval a během ekonomického růstu málo investoval. Další aktuální ekonomický a společenský problém je rostoucí chudoba a bída. Přes milion lidí je v exekucích a reálná mzda klesá. Petr Fiala odmítá, že by někoho nechala padnout. Prostřednictvím programu Milostivé léto se řada lidí s dluhy a exekucemi mělo možnost těchto dluhů zbavit. Zároveň se vláda snaží poskytovat sociální pomoc těm, kteří jí skutečně potřebují. Rovněž se snaží pomáhat uprchlíkům z Ukrajiny, kteří utekli z Ukrajiny po Ruské invazi. Jaké jsou hlavní domácí problémy České republiky, s nimiž se vládní koalice složená z koalic SPOLU, Pirátů a Starostů pod vedením premiéra Petra Fialy, musí vypořádat? Jaký je současný stav státního deficitu a státního dluhu v České republice? Jaká opatření podniká Petr Fiala a jeho vláda k řešení rostoucího státního deficitu, a jaký je jejich přístup k omezování výdajů a daní? Jak vzniká státní dluh a jaké způsoby financování jsou používány (pokladniční poukázky, státní dluhopisy, půjčky atd.)?  Jaký je postoj Petra Fialy k výši státního dluhu, a jakým způsobem se vyjadřuje k dědictví po předchozí vládě Andreje Babiše? Jakými opatřeními vláda reaguje na rostoucí chudobu, bídu a exekuce v zemi? Jak funguje program Milostivé léto a jaké výsledky dosáhl v řešení dluhů a exekucí občanů? Jaká opatření jsou přijímána k poskytování sociální pomoci potřebným a jak se vláda snaží řešit rostoucí chudobu? Jaký je postoj a opatření vlády vůči uprchlíkům z Ukrajiny, kteří utíkají po Ruské invazi? Jakým způsobem vláda podporuje ekonomický růst a investice do země? Jaký je plán Petra Fialy a jeho vlády na udržení hospodářské stability a sociální soudržnosti v budoucnosti? Jaký je celkový postoj veřejnosti k opatřením a politice Petra Fialy a jeho vlády v těchto oblastech? Chudoba. Krize. Dluh. Deficit. Rozpočet. Zadlužení. Stát."  

In [114]:
text1EMB = "The war in Ukraine is a conflict that began on February 24, 2022 with a large-scale invasion of Russian troops into Ukraine in an attempt to take control of the country within a few days. Ukraine successfully defended itself and partially pushed back the Russian army. Many European countries and the US condemned the Russian military attack and they are helping Ukraine with arms import, humanitarian aid and also accepting many refugees from Ukraine. Many economic sanctions have been imposed on Russia. Russia is also waging a hybrid war by spreading lies on the Internet through social networks, especially in the Central Europe region. Russian aggression has convinced neutral countries like Finland and Sweden to join NATO. Ukraine has been led in the war from the beginning by President Zelenensky. On the other side was Russian President Vladimir Putin. Both Russian and Ukrainian forces created a series of trench lines to defend the captured territories, which meant slowing down the offensive. Although President Zelenskyy shows his gratitude to the Western countries, he keeps saying that Ukraine receives only enough weapons to defend itself, but not enough to defeat Russia. The Western world is afraid of the nuclear weapons that Russia possesses. Prime Minister Petr Fiala approves the aid that is being sent to Ukraine. He was one of the first European politicians to visit President Zelensky in Kyiv, Ukraine. The current Czech government is still one of the most pro-Ukrainian and anti-Russian governments in the world, although the popularity of supporting Ukraine in society is declining. Petr Fiala is criticized for supporting Ukrainians and Ukrainian refugees at the expense of Czech citizens. However, he refuses. He is a supporter of increasing the budget of the army and the Ministry of Defense. New F-35 fighter jets were purchased during his administration. Fiala's opponents criticize the purchase as fiscally irresponsible. What was the original impetus to provoke the conflict between Russia and Ukraine in February 2022? How did Ukraine successfully defend itself against the initial invasion by Russian forces? Describe the international response to Russia's actions towards Ukraine. How did European countries and the United States support Ukraine in response to the Russian invasion? What economic sanctions have been imposed on Russia in connection with the conflict? Explore the role of social media in the hybrid warfare waged by Russia in Central Europe. Discuss the military strategies that were used by both Russian and Ukrainian forces, including the creation of trench lines for defense. What was the effect of these trench lines on the speed of the advance of the offensive? What is President Zelensky's attitude towards Western aid and what does he say about the amount of weapons provided? What is the attitude of Russian President Vladimir Putin towards the course of the war and the international reaction? How does Prime Minister Petr Fiala comment on the aid provided to Ukraine and what is his position on the purchase of new F-35 fighter jets? What is the current attitude of the Czech public towards the support of Ukraine, and what is the trend of the popularity of this support? What is Prime Minister Petr Fiala's approach to criticism that he allocates support to Ukraine at the expense of Czech citizens? What are the criticisms of the purchase of new F-35 fighter jets and what do its opponents say about budgetary responsibility? Ukraine. Ukraine. of Ukraine. Russia. War. Russian. Russians. to the Russians. War. Operation. Special Operations. Ukrainians."
text2EMB = "The conflict between Israel and Hamas began on October 7, 2023, when thousands of rockets were fired from the Gaza Strip aimed at Israel, and about 3,000 armed terrorists breached the wall separating Israel and Gaza and attacked nearby villages, looting, raping and killing. Back they took hundreds of hostages back to Gaza with them. Israel responded by cutting off Gaza from water and supplies and began firing rockets into Gaza. A few weeks later, Israel launched an invasion of Gaza. Hamas is trying to exchange hostages for Palestinians held in Israeli prisons. Late November a truce was held for several days. There were many protests in Europe and Asia for the Palestinian people for Israel to stop the war and allow humanitarian aid to Gaza. So far Israel has allowed some aid from many international humanitarian organizations. To date over ten thousand people have died in Palestine , mostly children. Social support for Israel in the Czech Republic is very strong. Petr Fiala expressed support for the Israeli government and emphasized the Israeli people's right to self-defense. The Czech Republic is a long-term partner of Israel and Petr Fiala is determined to continue this tradition. He calls Hamas a terrorist organization. Petr Fiala's left-wing opponents criticize him for ignoring the civilian casualties caused by Israel's bombing of the Gaza Strip. A number of countries send military aid to Israel. When and how did the conflict between Israel and Hamas start in 2023? What was the nature of the first attacks by Hamas and how did Israel respond to them? What were the consequences of the Israeli invasion of the Gaza Strip? Why did Hamas take hundreds of hostages in an attack on Israeli villages and what was their fate? How did Israel respond to the rocket fire and how did the conflict develop in the following weeks? What was Israel's goal in cutting off the Gaza Strip from water and supplies? Why is Hamas seeking to exchange hostages for Palestinians in prisons in Israel? How was the ceasefire at the end of November and how did it affect the conflict? What is the international response to the conflict in the Middle East, especially in Europe and Asia? What is the social support for Israel in the Czech Republic and what is the attitude of Prime Minister Petr Fiala towards this conflict? Why does he call Hamas a terrorist organization and what kind of criticism does he face from Petr Fiala's left-wing opponents? What is the Czech Republic's long-term relationship with Israel and how does Petr Fiala want to continue this tradition? What aid has been provided to the Palestinian people and how is Israel perceived by international humanitarian organizations? What is the state of the victims of the conflict on the Palestinian side and how is the situation developing in the Gaza Strip? Palestine. Palestine. Israel. Israel. War. Middle East. Iran. Arabs. Muslims. Muslim. Jews. Jews. "
text3EMB = "Presidential elections were held in the Czech Republic in January 2023. The first round was won by Petr Pavel, the former chairman of the NATO military committee, Petr Pavel was a pro-NATO, pro-European and pro-Western candidate. Prime Minister Petr Fiala showed his support for Petr Pavel. A populist candidate finished in close second place , former Prime Minister Andrej Babiš, who expressed disapproval of Czech participation in the war in Ukraine and the government's inability to care for the people. The second round was held on January 27 and 28. Andrej Babiš was an opponent of Petr Fiala. Petr Pavel won the second round against Babiš with with a significant margin of votes and became the newly elected president of the Czech Republic. He took office in the spring of 2023 and replaced Miloš Zeman. The election saw the highest voter turnout since 1998. Petr Fiala shares pro-Western values and views on Ukraine with President Petr Pavlo , which was attacked by Russia. The TOGETHER coalition led by Petr Fiala supported Pavlov's candidacy. What were the main characteristics of the presidential elections in the Czech Republic in January 2023? Who won the first round of elections and what were the main points of his political program? What attitude did Prime Minister Petr Fiala take towards the winner of the first round of elections? Who was the second most successful candidate in the first round and what was his attitude towards Czech participation in the war in Ukraine? What opinions did Andrej Babiš express regarding the government's ability to care for citizens? How did the second round of elections go and what was the result of the fight between Petr Pavlo and Andrej Babiš? What values and opinions about Ukraine does President Petr Pavel share with Prime Minister Petr Fiala? How was the voter turnout in the election and were there any specific aspects of this election race? What was the position of the SPOLU coalition, led by Petr Fiala, towards the presidential candidacy of Petr Pavel? When did Petr Pavel take over the presidency and who did he replace as president? What is the relationship between President Petr Pavlo and the previous president Miloš Zeman? What were the main points of President Petr Pavel's political program and how does it differ from the previous president? What was the general public reaction to the election result and the change of president? How is the change in the presidency expected to affect the Czech political scene and international relations? President. Presidential. Elections. Peter Paul. Peter. Pavel."
text4EMB = "Slovak parliamentary elections were held in September 2023 after the previous government of Igor Matovič lost a vote of no confidence. The winner of the election, left-wing populist and pro-Russian politician Robert Fico managed to form a government with the social democratic party HLAS and the Slovak National Party. Robert Fico led the campaign by spreading disinformation, lies and criticizing the European government, Ukraine and the ideas of liberal democracy. During the last few years, investigators have been busy uncovering many corrupt deals that were carried out under Fico's previous administrations. Petr Fiala and Fico do not share views on the country's pro-Russian orientation. After but on election night he congratulated Fico and wished Slovakia a 'good government' in the hope that the Czech and Slovak Republics will maintain their excellent relations, which is not entirely certain. There is a threat of Slovakia turning to and even more leaning towards Russia, which Petr Fiala criticizes for waging war in Ukraine . What was the reason for holding the Slovak parliamentary elections in September 2023 and what were the results of the voting? Who was the winner of the elections and what political coalition was formed? How did Robert Fico conduct the election campaign and what topics did he work with? What were the main elements of Fico's criticism of the European government, Ukraine and liberal democracy during the campaign? How did Robert Fico deal with corruption allegations linked to his previous government? What is Petr Fiala's attitude towards the pro-Russian orientation of Slovakia and what are the concerns regarding a possible turn towards Russia or Hungary? What was Petr Fialův's attitude towards Robert Fico's victory and what did he wish for him after the election results? What are the hopes for maintaining good relations between the Czech Republic and the Slovak Republic after the change in the Slovak government? What are the possible consequences for the region if Slovakia really reorients its policy towards Russia? How do you assess the situation when a left-wing populist politician with pro-Russian views gains leadership in the government? What impact can Slovakia's turn to the east have on stability and security in the region? How can these events affect Slovakia's attitude to the war in Ukraine and relations with Russia? What is the public reaction to politics and election results in Slovakia? What was the impact of this political change on the international scene and relations between neighboring countries in Central Europe? Fico. Matovič. Direction. Olano. Fic."
text5EMB = "Petr Fiala, born on September 1, 1964 in Brno, is a Czech politician, political scientist and university teacher, as of November 2021 the Prime Minister of the Czech Republic at the head of the majority cabinet formed by the coalition entities SPOLU and Pirates and Starostové. He has represented the ODS party in the Chamber of Deputies since 2013. He is currently the chairman of the ODS party and leads the SPOLU coalition, composed of the right-wing ODS, Christian Lidovci and the TOP 09 party. He served as Minister of Education in the Nečas cabinet in 2012 and 2013. Until 2011, he was rector and then vice-rector of Masaryk University in Brno. He comes from a Moravian bourgeois family. Grandfather František Fiala was a lawyer who, during the First Republic, worked as a high councilor of the political administration at the district hetmanships in Ostrava, Hodonín and Olomouc, and later as a state councilor in Brno. Paternal grandmother Františka Fialová was Jewish and part relatives were deported to German concentration camps during the war.In 1992, Petr Fiala married Jana Fialová, whom he met during the Velvet Revolution. The wife is a biologist working at the Institute of Health Protection and Promotion of the Faculty of Medicine of Masaryk University. In May 2023, she was appointed vice-rector of Masaryk University. Three children were born to the marriage. The eldest, Martin Fiala, studied art history at the MU Faculty of Arts, Klára Fialová became a student of medicine at the MU Faculty of Arts, and the youngest, Jiří Fiala, began studying history and economics. What are the key life facts about Petr Fial, Czech politician and chairman of the ODS? When did Petr Fiala start his political career and what positions did he hold before becoming Prime Minister in 2021? What political alliances did Petr Fiala lead before his presidency and what party coalition did his government create in 2021? What role did Petr Fiala play as Minister of Education in Nečas's cabinet in 2012 and 2013? What is his relationship with Masaryk University in Brno and what functions did he hold at this university? What was his family background and what are some interesting facts about his grandfather František Fial and grandmother Františka Fial? What is the history of Petr Fiala's marriage to Jana Fialová and what professions do they both hold? Who are Petr Fiala's children and what fields of study do they study? In what way is Petr Fiala's family involved in society and the academic environment? What was the public reaction to the appointment of Jana Fialová as vice-rector of Masaryk University in May 2023? What was Petr Fiala's contribution in the field of political science and what are his main political priorities? What are the prospects for Petr Fiala's political career in the future? What was the development of the ODS party under the leadership of Petr Fiala and how is his political style reflected in party activities? What are the main challenges Petr Fiala faces in his political career and as head of the Czech government? When you were born? What is your wife's name? What are your children's names? Where do your children go to school? What did you study?"
text6EMB = "Among the main domestic problems of the Czech Republic, with which the government coalition consisting of SPOLU and the Pirates and Starosts coalitions led by Prime Minister Petr Fiala, must deal with, is the growing state deficit. The state debt has risen to 3.2 trillion crowns. Petr Fiala is trying to reduce state spending budget and at the same time not to raise taxes, as he promised in his program. The national debt is made up of the government's debts and arises primarily from the accumulation of state budget deficits. It is financed by treasury bills, government bonds, direct loans or loans from the European Investment Bank. Petr Fiala says that the government's high indebtedness inherited from the government of Andrej Babiš and his irresponsible fiscal policy. Fiala claims that Babiš squandered unnecessarily and invested little during economic growth. Another current economic and social problem is growing poverty and misery. Over a million people are in foreclosures and the real wage is falling. Petr Fiala refuses to let anyone down.Through the Summer of Grace program, many people with debt and foreclosures have had the opportunity to get out of debt. At the same time, the government tries to provide social assistance to those who really need it. They also try to help refugees from Ukraine who fled Ukraine after the Russian invasion. What are the main domestic problems of the Czech Republic that the government coalition consisting of SPOLU, Pirates and Starosts under the leadership of Prime Minister Petr Fiala must deal with? What is the current state of the state deficit and state debt in the Czech Republic? What measures are Petr Fiala and his government taking to address the growing state deficit, and what is their approach to spending and tax cuts? How is the national debt created and what financing methods are used (treasury bills, government bonds, loans, etc.)? What is Petr Fiala's position on the level of the national debt, and how does he express himself about the legacy of the previous government of Andrej Babiš? What measures is the government taking to address the growing poverty, misery and foreclosures in the country? How does the Gracious Summer program work and what results did it achieve in solving debts and foreclosures of citizens? What measures are being taken to provide social assistance to the needy and how is the government trying to address the growing poverty? What is the government's attitude and measures towards refugees from Ukraine who are fleeing after the Russian invasion? How does the government support economic growth and investment in the country? What is the plan of Petr Fiala and his government to maintain economic stability and social cohesion in the future? What is the general attitude of the public towards the measures and policies of Petr Fiala and his government in these areas? Poverty. Crisis. Debt. Deficit. Budget. Indebtedness. State."


In [135]:
contexts = {
    'ukraine': text1,
    'palestine_israel': text2,
    'president_elections': text3,
    'slovakia_elections': text4,
    'personal_information': text5,
    'debt_and_poverty': text6
}

contextsEMB = {
    'ukraine': text1EMB,
    'palestine_israel': text2EMB,
    'president_elections': text3EMB,
    'slovakia_elections': text4EMB,
    'personal_information': text5EMB,
    'debt_and_poverty': text6EMB
}



In [78]:
q_ukraine = [
    "Kdy začala konflikt na Ukrajině a jaký byl jeho původ?",
    "Jaký byl vliv anexe Krymu na eskalaci konfliktu?",
    "Které země podporují Ukrajinu a které podporují separatisty?",
    "Jaká je aktuální humanitární situace v postižených oblastech?",
    "Jaké jsou diplomatické snahy o řešení konfliktu a mírový proces?",
    "Jaký je vliv války na civilní obyvatelstvo v dané oblasti?",
    "Které mezinárodní organizace se aktivně angažují ve snaze ukončit konflikt?",
    "Jaké jsou následky války na ekonomiku Ukrajiny a okolních zemí?",
    "Existuje nějaký plán obnovy a rekonstrukce po skončení konfliktu?",
    "Jaký je postoj veřejnosti v různých částech světa k ukrajinskému konfliktu?"
]

# Válka v Palestině
q_palestine = [
    "Jak vznikl současný konflikt mezi Izraelem a Palestinou?",
    "Které faktory přispěly k nedávným násilnostem a eskalaci napětí?",
    "Jaké jsou hlavní body sporu mezi Izraelem a Palestinci?",
    "Jaké jsou důsledky konfliktu na civilní obyvatelstvo na obou stranách?",
    "Jaká je role mezinárodní komunity při řešení izraelsko-palestinského konfliktu?",
    "Existuje nějaký mírový plán nebo jednání mezi stranami konfliktu?",
    "Jaký je postoj arabských států k izraelsko-palestinskému konfliktu?",
    "Které mezinárodní organizace monitorují situaci a poskytují humanitární pomoc?",
    "Jak se promítá konflikt do regionální stability na Blízkém východě?",
    "Jaký má konflikt vliv na vnímání Izraele a Palestiny ve světě?"
]

# České prezidentské volby
q_president = [
    "Kdo jsou hlavní kandidáti v nadcházejících českých prezidentských volbách?",
    "Jaké jsou hlavní politické témata diskutovaná během volební kampaně?",
    "Jaké jsou klíčové výzvy, kterým bude český prezident čelit v nadcházejícím období?",
    "Jaký je postoj veřejnosti k jednotlivým kandidátům a jejich programům?",
    "Jaký je vliv mediálního pokrytí na průběh volební kampaně?",
    "Jakým způsobem probíhá volební proces a jaký je očekávaný volební účast?",
    "Které politické strany nebo hnutí podporují jednotliví kandidáti?",
    "Jaká je role prezidenta v českém politickém systému a jaký má pravomoci?",
    "Jaké jsou předpokládané důsledky výsledků prezidentských voleb pro politickou scénu v České republice?",
    "Jaký je vliv prezidentských voleb na vnímání České republiky v mezinárodním kontextu?"
]

# Slovenské parlamentní volby
q_slovakia = [
    "Kdo jsou hlavní politické strany a kandidáti ve slovenských parlamentních volbách?",
    "Jaké jsou klíčové politické otázky, které dominují volební kampani?",
    "Jaký je vztah mezi Slovenskem a Evropskou unií v kontextu voleb?",
    "Jaký je postoj veřejnosti k jednotlivým politickým stranám a lídrům?",
    "Jaký je politický systém na Slovensku a jak funguje parlamentní volba?",
    "Které otázky týkající se ekonomiky a sociální politiky jsou ve volbách klíčové?",
    "Jaká je role Slovenska v regionálním a mezinárodním kontextu, a jak ovlivňují volby tuto pozici?",
    "Jaká je očekávaná volební účast a jaké jsou faktory, které ji mohou ovlivnit?",
    "Jaký je postoj kandidátů k otázkám životního prostředí a udržitelnosti?",
    "Jak by výsledek voleb mohl ovlivnit vnitřní politické a hospodářské události ve Slovenské republice?"
]

In [15]:
model_name = "ufal/robeczech-base" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

In [117]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [92]:
# tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/bert-base-bg-cs-pl-ru-cased')
# model = AutoModel.from_pretrained('DeepPavlov/bert-base-bg-cs-pl-ru-cased')

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/bert-base-bg-cs-pl-ru-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [118]:
feature_extraction_pipeline = pipeline('feature-extraction', model=model, tokenizer=tokenizer)  # 'device=0' for GPU, remove for CPU

In [139]:
# json_file_path = "/home/martin/Coding/DAS-political-confesions/embedding/stopwords-cs.json"
# # Open and load the JSON file
# with open(json_file_path, "r", encoding="utf-8") as file:
#     stopwords = json.load(file)
    
# special_characters_mapping = {'š': 's', 'č': 'c', 'ě': 'e', 'ž': 'z', 'ř': 'r', 'ý': 'y', 'á': 'a', 'í': 'i', 'é': 'e', 'ň': 'n'}
# special_characters_mapping.update({char.upper(): replacement.upper() for char, replacement in special_characters_mapping.items()})

def preprocess_text(text, stopwords):
    
#     for special_char, replacement in special_characters_mapping.items():
#         text = text.replace(special_char, replacement)
    
    words = text.split()

    words = [word[0:5] for word in words if len(word) > 7]
    
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

import nltk
from nltk.corpus import stopwords

# Download the stopwords dataset
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stop_words(sentence):
    # Tokenize the sentence and remove stop words
    filtered_tokens = [word for word in sentence.split() if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)


[nltk_data] Downloading package stopwords to /home/martin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [145]:
embeddings = {}
for key, text in contextsEMB.items():
    
#     text = preprocess_text(text, stopwords)
    text = remove_stop_words(text)
    chunk_size = 300  # Set your desired chunk size

    # Split the text into non-overlapping chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Initialize an empty list to store chunk embeddings
    chunk_embeddings = []

    # Process each chunk and obtain embeddings
    for chunk in chunks:
        chunk_embedding = feature_extraction_pipeline(chunk)
        chunk_embedding = np.mean(chunk_embedding[0], axis=0)
        chunk_embeddings.append(chunk_embedding)

    # Aggregate embeddings (e.g., by averaging)
    aggregated_embedding = np.mean(chunk_embeddings, axis=0)
    aggregated_embedding = np.array(aggregated_embedding).reshape(1, -1)
    embeddings[key] = aggregated_embedding
    

In [141]:

question_embeddings = {}
for q in q_slovakia:
#     q = preprocess_text(q, stopwords)
    translated_q = GoogleTranslator(source='cs', target='en').translate(q)
    q = remove_stop_words(translated_q)

#     question = preprocess_text(question, stopwords)
    emb_q = feature_extraction_pipeline(q)
    emb_q = np.mean(emb_q[0], axis=0)
    emb_q = np.array(emb_q).reshape(1, -1)
    question_embeddings[q] = emb_q
    


In [146]:
question_classes = {}

for q, emb_q in question_embeddings.items():
    max_sim = -1
    for i,(t, emb_t) in enumerate(embeddings.items()):
        sim = cosine_similarity(emb_q, emb_t)
        print(q, "  ", i, "  ", sim)
        if sim > max_sim:
            question_classes[q] = i
            max_sim = sim
    print(question_classes[q], "  ", max_sim)
        

main political parties candidates Slovak parliamentary elections?    0    [[0.30243639]]
main political parties candidates Slovak parliamentary elections?    1    [[0.21617586]]
main political parties candidates Slovak parliamentary elections?    2    [[0.63666373]]
main political parties candidates Slovak parliamentary elections?    3    [[0.66640065]]
main political parties candidates Slovak parliamentary elections?    4    [[0.49327759]]
main political parties candidates Slovak parliamentary elections?    5    [[0.30356982]]
3    [[0.66640065]]
key political issues dominating election campaign?    0    [[0.29977211]]
key political issues dominating election campaign?    1    [[0.23424369]]
key political issues dominating election campaign?    2    [[0.40374429]]
key political issues dominating election campaign?    3    [[0.36800665]]
key political issues dominating election campaign?    4    [[0.24625013]]
key political issues dominating election campaign?    5    [[0.28103796]]
2 

In [147]:
for key, value in question_classes.items():
    print(key, ': ', value)

main political parties candidates Slovak parliamentary elections? :  3
key political issues dominating election campaign? :  2
relationship Slovakia European Union context elections? :  3
public's attitude towards individual political parties leaders? :  2
political system Slovakia parliamentary election work? :  3
economic social policy issues key election? :  5
role Slovakia regional international context, elections affect position? :  3
expected turnout factors affect it? :  2
position candidates environmental sustainability issues? :  4
could result election affect internal political economic events Slovak Republic? :  3


In [148]:
import pickle

In [149]:
with open('contexts_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [136]:
with open('contexts.pkl', 'wb') as f:
    pickle.dump(contexts, f)

In [109]:
from deep_translator import GoogleTranslator
to_translate = 'Co si myslíš o konfliktu na Ukrajině?'
translated = GoogleTranslator(source='cs', target='en').translate(to_translate)

In [110]:
print(translated)

What do you think about the conflict in Ukraine?
